In [1]:
from google.colab import drive
drive.mount('/content/drive/')
DIR='/content/drive/MyDrive/Research/OntoConnectWithGNN/GNN_1/'

import os
os.chdir(DIR)

Mounted at /content/drive/


In [2]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'''))

!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

!pip install git+https://github.com/facebookresearch/fastText.git
!pip install git+https://github.com/facebookresearch/PyTorch-BigGraph.git

<IPython.core.display.Javascript object>

1.7.0+cu101
10.1
  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-mdpkv0ic
  Running command git clone -q https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-mdpkv0ic
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3088331 sha256=5b6b2fa6ad7a5aa3cb93464fdb86d6844385c2f6217749c514a0d366d94f8f29
  Stored in directory: /tmp/pip-ephem-wheel-cache-ixxu1ten/wheels/69/f8/19/7f0ab407c078795bc9f86e1f6381349254f86fd7d229902355
Successfully built fasttext
  Cloning https://github.com/facebookresearch/PyTorch-BigGraph.git to /tmp/pip-req-build-6tuvgy0m
  Running command git clone -q https://github.com/facebookresearch/PyTorch-BigGraph.git /tmp/pip-req-build-6tuvgy0m
  Created wheel for torchbiggraph: filename=torchbiggraph-1.0.1.dev0-cp37-none-any.whl size=118953 sha256=41b1c40c97d072fc98630b5602e79dc40009d87e6e831d01d1dba9f1c8ade11c
  Stored in directory: /tmp/pip-ephem-wheel-cache-h_mhvo82/wheels/d5/d0/e8/9fa5

In [3]:
#from OntoSimImports import *
%run OntoSimImports.py

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
#import OntoSimConstants
%run OntoSimConstants.py
from OntoSimConstants import *

In [5]:
def assignVar():

    conf = {
        "conf_arr": [
            {
                'ind' : 'source',
                'ip_fl_nm': DATA_DIR+'fastentity/source_fast.json',
                'op_fl_path': DATA_DIR+'gnnentity/source_gnn.json'
            },
            {
                'ind' : 'target',
                'ip_fl_nm': DATA_DIR+'fastentity/target_fast.json',
                'op_fl_path': DATA_DIR+'gnnentity/target_gnn.json'
            }
        ]
    }

    return conf

In [6]:
def loadFile(conf):
    fl_nm = code_path+conf['ip_fl_nm']
    with open(fl_nm) as f:
        data = json.load(f)

    return data

In [7]:
def populateEdgesPerEntity(entity_obj, edges):
    centerNode=entity_obj['iri']
    edges.append([centerNode, 'self', centerNode])

    if(entity_obj['parentCls']):
      for parent in entity_obj['parentCls']:
        edges.append([centerNode, 'parent', parent])
        edges.append([parent, 'parent', centerNode])

    if(entity_obj['childCls']):
      for child in entity_obj['childCls']:
        edges.append([centerNode, 'child', child])
        edges.append([child, 'child', centerNode])

    if(entity_obj['eqCls']):
      for eq in entity_obj['eqCls']:
        edges.append([centerNode, 'equivalent', eq])
        edges.append([eq, 'equivalent', centerNode])

    if(entity_obj['disjointCls']):
      for disjoint in entity_obj['disjointCls']:
        edges.append([centerNode, 'disjoint', disjoint])
        edges.append([disjoint, 'disjoint', centerNode])

    if(entity_obj['restriction']):
      for restriction in entity_obj['restriction']:
        expression = restriction.replace("(", "( ").replace(",", " , ").replace(")", " ) ")
        tokens = expression.split(" ")
        token_lst = []
        for token in tokens:
          if("" != token and " " != token and "," != token):
            token_lst.append(token)
        edges.append([centerNode, 'restriction', token_lst[-2]]) #this needs tobe changed with SHUNTING YARD ALGORITHM
        edges.append([token_lst[-2], 'restriction', centerNode])

    return edges

In [8]:
def saveEdgePerEntity(entity_info, conf_val):
    with open(code_path+conf_val["op_fl_path"], 'w') as outfile:
        json.dump(entity_info, outfile, indent=4)

In [9]:
def populateAndSaveEdges(conf_val, entity_info):
  for entity in entity_info:
    entity_obj=entity_info[entity]
    edges = []
    edges = populateEdgesPerEntity(entity_obj, edges)
    entity_obj['graphEdges']=edges
    entity_info[entity]=entity_obj
    
  saveEdgePerEntity(entity_info, conf_val)

In [10]:
#################### MAIN CODE START ####################
def populateEdges():
    try:
        print("#################### PopulateEdges START ####################")
        conf = assignVar()
        #####Mean of vectors
        conf_arr = conf["conf_arr"]
        for conf_val in conf_arr:
            entity_info = loadFile(conf_val)
            populateAndSaveEdges(conf_val, entity_info)
            time.sleep(wait_time)

    except Exception as exp:
        raise exp
    finally:
        print("#################### PopulateEdges FINISH ####################")

#################### MAIN CODE END ####################

In [11]:
### MAIN FUNCTION
if __name__=="__main__":
  populateEdges()

#################### PopulateEdges START ####################
#################### PopulateEdges FINISH ####################
